In [ ]:
#리뷰 요약테이블 저장 공간
"""
반환값 : 없음 그냥 호출되면 리뷰 요약 테이블에 db저장하고 끝
"""
def review_sum_server(comp_uid,sum):
    pass

In [1]:
import sys
sys.path.append('/app/EZPZ_Project/modules/torchmodules') 
import tokenizer as tk

In [2]:
sents = ['안녕하세요 제 이름은 김동현이라고 합니다, 저는 플레이데이터 수강중입니다.', '안녕! 나는 김동현이야. 나는 플레이데이터 수강중이야.']
tk.get_keyword_nng(sents, 'review')